In [2]:
import pandas as pd
df_decision_with_reasoning = pd.read_csv('../data/final_test/final/reexperiment/fewhot/11August/df_decision_with_reasoning.csv', index_col=False)

In [1]:
import pandas as pd

df_accepted = pd.read_csv('../data/extraction_results/df_accepted.csv', index_col=False)

In [2]:
def generate_caselaw_url(para_id: str) -> str:
    """
    Generate caselaw URL from para_id.
    Example: ewfc_b_2024_40#para_77 -> https://caselaw.nationalarchives.gov.uk/ewfc/b/2024/40
    """
    try:
        if '#' in para_id:
            case_id = para_id.split('#')[0]
        else:
            case_id = para_id
        
        # Convert underscores to forward slashes for URL
        url_path = case_id.replace('_', '/')
        return f"https://caselaw.nationalarchives.gov.uk/{url_path}"
    except Exception:
        return ""
def parse_legislation_id(section_id: str) -> str:
     # Default to returning the original section_id if parsing fails
    """
    Parse section_id to extract standardized legislation ID.
    Examples:
    - http://www.legislation.gov.uk/id/ukpga/1990/8/section/73_ -> 1990/8_73
    - http://www.legislation.gov.uk/id/ukpga/1990/8/section/73 -> 1990/8_73
    - id/ukpga/1990/8_section-191 -> 1990/8_191
    - id/ukpga/1989/41/section/31 -> 1989/41_31
    """
    try:
        # Remove trailing underscore if present
        section_id = section_id.rstrip('_')
        
        # Remove URL prefix if present
        if section_id.startswith('http://www.legislation.gov.uk/'):
            section_id = section_id.replace('http://www.legislation.gov.uk/', '')
        
        # Handle different formats
        if '/section/' in section_id:
            # Format: id/ukpga/1990/8/section/73
            parts = section_id.split('/')
            

    
            if len(parts) >= 5 and parts[0] == 'id' and parts[1] == 'ukpga':
                year = parts[2]
                chapter = parts[3]
                section = parts[5] if len(parts) > 5 else parts[4]
                # Remove any trailing underscore from section number
                section = section.rstrip('_')
                return f"{year}/{chapter}_{section}"
        
        elif '_section-' in section_id:
            # Format: id/ukpga/1990/8_section-191
            parts = section_id.split('/')
            if len(parts) >= 3 and parts[0] == 'id' and parts[1] == 'ukpga':
                year = parts[2]
                chapter_section = parts[3] if len(parts) > 3 else parts[2]
                if '_section-' in chapter_section:
                    chapter, section = chapter_section.split('_section-')
                    # Remove any trailing underscore from section number
                    section = section.rstrip('_')
                    return f"{year}/{chapter}_{section}"
        
        # Fallback: try to extract year/chapter_section pattern
        match = re.search(r'(\d{4})/(\d+).*?(\d+)', section_id.rstrip('_'))
        if match:
            year, chapter, section = match.groups()
            return f"{year}/{chapter}_{section}"
            
        return section_id  # Return original if parsing fails
        
    except Exception:
        return section_id  # Return original if any error occurs
def create_standardized_act_id(section_id: str, section_number: str = "") -> str:
    """
    Create standardized act ID in format: year/chapter_section_subsection
    Example: 1989/41_section_91A
    """
    try:
        legislation_id = parse_legislation_id(section_id)
        if '_' in legislation_id:
            base_part = legislation_id.split('_')[0]  # e.g., "1989/41"
            section_part = legislation_id.split('_')[1] if '_' in legislation_id else ""
            return f"{base_part}_section_{section_part}"
        return f"{legislation_id}_section_{section_number}"
    except Exception:
        return section_id

def extract_para_id_from_full_id(para_id: str) -> str:
    """Extract just the paragraph ID from full identifier."""
    if '#' in para_id:
        return para_id.split('#')[-1]
    return para_id

In [3]:
df_accepted['standardized_act_id'] = df_accepted['section_id'].apply(create_standardized_act_id)
df_accepted['url'] = df_accepted['para_id'].apply(generate_caselaw_url)
df_accepted['para_id_new'] = df_accepted['para_id'].apply(extract_para_id_from_full_id)

In [ ]:
df_decision_with_reasoning['standardized_act_id'] = df_decision_with_reasoning['section_id'].apply(create_standardized_act_id)
df_decision_with_reasoning['url'] = df_decision_with_reasoning['para_id'].apply(generate_caselaw_url)
df_decision_with_reasoning['para_id_new'] = df_decision_with_reasoning['para_id'].apply(extract_para_id_from_full_id)

In [9]:
# Script to write the first 50 rows of df_no_duplicates to a text file in the specified format
output_file = "../data/final_test/final/reexperiment/fewhot/evaluation/accepted_3000-3100.txt"


with open(output_file, "w", encoding="utf-8") as f:
    for idx, row in df_accepted[3000:3100].iterrows():
        paralink = f"{row['url']}#{row['para_id_new']}"
        sectionlink = f"https://www.legislation.gov.uk/ukpga/{row['standardized_act_id'].replace('_','/')}"
        paragraph = row['para_text']
        section = row['section_text']
        case_law_excerpt = row['case_law_excerpt']
        legislation_excerpt = row['legislation_excerpt']
        reasoning = row['reasoning']
        model = row['model']
        f.write(f"**paralink** = {paralink}\n")
        f.write(f"**sectionlink** = {sectionlink}\n")
        f.write(f"**paragraph** = {paragraph}\n")
        f.write(f"**section** = {section}\n")
        f.write(f"**model** = {model}\n")
        f.write(f"**case_law_excerpt** = {case_law_excerpt}\n")
        f.write(f"**legislation_excerpt** = {legislation_excerpt}\n")
        f.write(f"**reasoning** = {reasoning}\n")
        f.write(f"**IS_Extracton_Correct (YES/No)** = \n")
        f.write(f"**IS_Reasoning_Correct (YES/No)** = \n")
        f.write("="*78 + "\n")


In [13]:
# Script to write the first 50 rows of df_no_duplicates to a text file in the specified format
output_file = "../data/final_test/final/reexperiment/fewhot/evaluation/sample_3000-3250.txt"


with open(output_file, "w", encoding="utf-8") as f:
    for idx, row in df_decision_with_reasoning[3000:3250].iterrows():
        paralink = f"{row['url']}#{row['para_id_new']}"
        sectionlink = f"https://www.legislation.gov.uk/ukpga/{row['standardized_act_id'].replace('_','/')}"
        paragraph = row['paragraphs']
        section = row['section_text']
        case_law_excerpt = row['case_law_excerpt']
        legislation_excerpt = row['legislation_excerpt']
        reasoning = row['reasoning']
        model = row['model']
        f.write(f"**paralink** = {paralink}\n")
        f.write(f"**sectionlink** = {sectionlink}\n")
        f.write(f"**paragraph** = {paragraph}\n")
        f.write(f"**section** = {section}\n")
        f.write(f"**model** = {model}\n")
        f.write(f"**case_law_excerpt** = {case_law_excerpt}\n")
        f.write(f"**legislation_excerpt** = {legislation_excerpt}\n")
        f.write(f"**reasoning** = {reasoning}\n")
        f.write(f"**IS_Extracton_Correct (YES/No)** = \n")
        f.write(f"**IS_Reasoning_Correct (YES/No)** = \n")
        f.write("="*78 + "\n")

# Script to read the file and calculate the score for IS_Extracton_Correct and IS_Reasoning_Correct











In [31]:
def calculate_score(filename, field):
    """
    Calculates the score for a given field (either 'IS_Extracton_Correct' or 'IS_Reasoning_Correct')
    in the evaluation file. Returns the number of YES, NO, and the percentage of YES.
    This function reads the file, splits it into records using the separator line,
    and then extracts the value for the specified field from each record.
    """
    yes_count = 0
    no_count = 0
    with open(filename, "r", encoding="ISO-8859-1") as f:
        content = f.read()
    # Split the content into records using the separator line
    records = [rec.strip() for rec in content.split("="*78) if rec.strip()]
    for record in records:
        for line in record.splitlines():
            try:
                if line.strip().startswith(f"**{field} (YES/No)**"):
                    # Get the value after the last '=' and strip spaces and asterisks
                    value = line.split("=")[-1].strip().replace("*", "").upper()
                    if value == "YES":
                        yes_count += 1
                    elif value == "NO":
                        no_count += 1
                    break  # Only one field per record
            except:
                pass
    total = yes_count + no_count
    percent_yes = (yes_count / total * 100) if total > 0 else 0
    return {"YES": yes_count, "NO": no_count, "Percent YES": percent_yes}

output_file = "../data/final_test/final/extraction-gpt4o-mini-swansea/for_validation/extraction_evaluation-50-100_COMPLETE.txt"

# Example usage:
extraction_score = calculate_score(output_file, "IS_Extracton_Correct")
reasoning_score = calculate_score(output_file, "IS_Reasoning_Correct")

print("IS_Extracton_Correct Score:", extraction_score)
print("IS_Reasoning_Correct Score:", reasoning_score)

IS_Extracton_Correct Score: {'YES': 43, 'NO': 7, 'Percent YES': 86.0}
IS_Reasoning_Correct Score: {'YES': 46, 'NO': 4, 'Percent YES': 92.0}


In [ ]:
# Make package for the failed cases to check the evaluation
#reasonA = The paragraph does not contain any "application of law"
#raasonB = The provided section is not which is applied to the particular paragraph or does not have any relevance to the paragraph
#reasonC = The system failed to extract the case term or legislation term with High confidence
import re
import pandas as pd
df_source_for_review = pd.read_csv('../data/final_test/final/reexperiment/fewhot/11August/df_source_for_review.csv', index_col=False)
output_file = "../data/final_test/final/reexperiment/fewhot/evaluation/failed_cases.txt"


df_source_for_review['standardized_act_id'] = df_source_for_review['section_id'].apply(create_standardized_act_id)
df_source_for_review['url'] = df_source_for_review['para_id'].apply(generate_caselaw_url)
df_source_for_review['para_id_new'] = df_source_for_review['para_id'].apply(extract_para_id_from_full_id)

with open(output_file, "w", encoding="utf-8") as f:
    f.write("**Failed Cases for Review**\n")
    f.write("reasonA = The paragraph does not contain any 'application of law'\n")
    f.write("raasonB = The provided section is not which is applied to the particular paragraph or does not have any relevance to the paragraph'\n")
    f.write("reasonC = The system failed to extract the case term or legislation term with High confidence\n")
    f.write("="*78 + "\n")
#reasonC = The system failed to extract the case term or legislation term with High confidence")
    for idx, row in df_source_for_review.iterrows():
        paralink = f"{row['url']}#{row['para_id_new']}"
        sectionlink = f"https://www.legislation.gov.uk/ukpga/{row['standardized_act_id'].replace('_','/')}"

        paragraph = row['paragraphs']
        section = row['section_text']

        f.write(f"**paralink** = {paralink}\n")
        f.write(f"**sectionlink** = {sectionlink}\n")
        f.write(f"**paragraph** = {paragraph}\n")
        f.write(f"**section** = {section}\n")

        f.write(f"**ReasonA (Yes/No)** = \n")
        f.write(f"**ReasonB (Yes/No)** = \n")
        f.write(f"**ReasonC (Yes/No)** = \n")
        f.write("="*78 + "\n")



In [14]:
def calculate_score(filename, field):
    """
    Calculates the score for a given field (either 'IS_Extracton_Correct' or 'IS_Reasoning_Correct')
    in the evaluation file. Returns the number of YES, NO, and the percentage of YES.
    This function reads the file, splits it into records using the separator line,
    and then extracts the value for the specified field from each record.
    """
    yes_count = 0
    no_count = 0
    with open(filename, "r", encoding="ISO-8859-1") as f:
        content = f.read()
    # Split the content into records using the separator line
    records = [rec.strip() for rec in content.split("="*78) if rec.strip()]
    for record in records:
        for line in record.splitlines():
            try:
                if line.strip().startswith(f"**{field} (Yes/No)**"):
                    # Get the value after the last '=' and strip spaces and asterisks
                    value = line.split("=")[-1].strip().replace("*", "").upper()
                    if value == "YES":
                        yes_count += 1
                    elif value == "NO":
                        no_count += 1
                    break  # Only one field per record
            except:
                pass
    total = yes_count + no_count
    percent_yes = (yes_count / total * 100) if total > 0 else 0
    return {"Yes": yes_count, "No": no_count, "Percent Yes": percent_yes}


# Example usage:
reasoning_a_score = calculate_score(output_file, "ReasonA")

reasoning_b_score = calculate_score(output_file, "ReasonB")
reasoning_c_score = calculate_score(output_file, "ReasonC")

print("ReasonA Score:", reasoning_a_score)
print("ReasonB Score:", reasoning_b_score)
print("ReasonC Score:", reasoning_c_score)

ReasonA Score: {'Yes': 4, 'No': 2, 'Percent Yes': 66.66666666666666}
ReasonB Score: {'Yes': 1, 'No': 2, 'Percent Yes': 33.33333333333333}
ReasonC Score: {'Yes': 0, 'No': 3, 'Percent Yes': 0.0}
